# **05-opera-api.ipynb**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from cartopy.feature import BORDERS, COASTLINE

In [ ]:
from smash.processing.hailtrack import apply_advection_correction
from smash.service.download.opera import download_files
from smash.service.read.composite import get_opera_projection_cfg, open_opera_dataset
from smash.tools.common_path import SOURCE_DATA_PATH

## Download

In [ ]:
VERBOSE = True

In [ ]:
time_bounds = {
    "start": "2021-07-14",
    "end": "2021-07-14"
}
product = "REFLECTIVITY"
local_path = SOURCE_DATA_PATH / "opera"
local_path.mkdir(parents=True, exist_ok=True)

In [ ]:
status = download_files(time_bounds, product, local_path)

## Get files

In [ ]:
files = status["downloaded"][:20]

## Read

In [ ]:
projection_cfg = get_opera_projection_cfg()

In [ ]:
ds = xr.concat(
    [open_opera_dataset(file, projection_cfg) for file in files],
    dim="time"
)

## Plot

In [ ]:
if VERBOSE:
    ax = plt.axes(projection=ccrs.PlateCarree())
    ds.sel(time="2021-07-14 00:15").data.plot(
        x="longitude",
        y="latitude",
        ax=ax,
        vmin=0,
        vmax=70,
        cmap="gist_ncar",
        cbar_kwargs={"label": "Reflectivity (dBZ)"}
    )
    ax.add_feature(COASTLINE)
    ax.add_feature(BORDERS)
    gls = ax.gridlines(draw_labels=True, alpha=0.5, linestyle="--")
    gls.top_labels, gls.right_labels = False, False
    ax.set(xlim=[0, 10], ylim=[45, 53])
    ax.set_aspect("auto", adjustable=None)

## Advection Correction

- compute

In [ ]:
coordinates_box = {
    "latitude": [49, 53],
    "longitude": [5, 9.5]
}

In [ ]:
ds_smaller = ds.where(
    (ds.latitude >= min(coordinates_box["latitude"])) &
    (ds.latitude <= max(coordinates_box["latitude"])) &
    (ds.longitude >= min(coordinates_box["longitude"])) &
    (ds.longitude <= max(coordinates_box["longitude"])),
    drop=True
).fillna(0)

In [ ]:
ds_smaller_advected = apply_advection_correction(
    ds_smaller,
    interpolation_timestep=pd.Timedelta("5min")
)

- plot

In [ ]:
if VERBOSE:
    fig, axs = plt.subplots(
        1, 2, figsize=[14, 5], subplot_kw={'projection': ccrs.PlateCarree()}
    )
    ds_smaller.data.max("time").plot(
        x="longitude", y="latitude", ax=axs[0], vmin=0, vmax=70, cmap="gist_ncar",
    )
    axs[0].set(title="Raw Cumulative Reflectivity (T=15min)")
    axs[0].add_feature(COASTLINE)
    axs[0].add_feature(BORDERS)
    gls0 = axs[0].gridlines(draw_labels=True, alpha=0.5, linestyle="--")
    gls0.top_labels, gls0.right_labels = False, False
    
    ds_smaller_advected.data.plot(
        x="longitude", y="latitude", ax=axs[1], vmin=0, vmax=70, cmap="gist_ncar",
    )
    axs[1].set(title="Advected Cumulative Reflectivity (t=5min)")
    axs[1].add_feature(COASTLINE)
    axs[1].add_feature(BORDERS)
    gls1 = axs[1].gridlines(draw_labels=True, alpha=0.5, linestyle="--")
    gls1.top_labels, gls1.right_labels = False, False


---
## End of the notebook
---